In [1]:
# import packages

import os # read system path 
import csv

import matplotlib as mpl
import matplotlib.pyplot as plt
import librosa
import librosa.display

import pandas as pd
from scipy import signal
import soundfile as sf
from gudhi.point_cloud import timedelay
import numpy as np
from numpy import argmax
import math
from ripser import ripser
from persim import plot_diagrams

import umap
from sklearn.preprocessing import StandardScaler
%matplotlib qt5

# Path is where the voiced/voicedless wav file located
voicedPath="/Users/pfeng3/Documents/research/TopCap/revise2/audio_segment/voiced/"
voicedlessPath="/Users/pfeng3/Documents/research/TopCap/revise2/audio_segment/voiceless/"

# function define

# wav_fraction_finder is to find the corresponding wav signal according to interval
def wav_fraction_finder(start_time, end_time,sig):
    sig_fraction=sig[int(start_time*samplerate):int(end_time*samplerate)]
    return sig_fraction

# head_tail_scissor is to erase signal in head and tail that has amplitude smaller than 0.05
# can also use it to see if the length of renewing signal is greater than 500 or not 
def head_tail_scissor(sig):
    valid_interval=[index for index in range(len(sig)) if (sig[index]>0.03)]
    if len(valid_interval)==0:
        return False,sig
    head=min(valid_interval)
    tail=max(valid_interval)
    sig=sig[head:tail+1]
    if tail-head<500:
        return False,sig
    return True,sig

In [41]:
for fn in os.listdir(voicedPath):
    # Subsample dataset, retrieve 1 in 10 among dataset
    randNum=np.random.randint(10)
    if randNum !=0:
        continue

    # Read wav file as "sig"
    fileName,ext=os.path.splitext(fn)
    wavFile=voicedPath+fileName+".wav"
    sig,samplerate=sf.read(wavFile)

    # Cut head and tail of wav file is those sections are not reliable
    status,sig=head_tail_scissor(sig)
    if status==False:
        continue
    
    # Write result in a csv file
    with open("MFCC_Diag.csv","a",newline="") as csvfile:
        writer=csv.writer(csvfile)

        # MFCC
        mfccs = librosa.feature.mfcc(y=sig, sr=samplerate, n_mfcc=10, n_fft=int(len(sig)/2),hop_length=int(len(sig)/4))
        mfccs=np.abs(mfccs.flatten())

        # Add last feature to indicate if it is voiced/ voicedless
        # 0 indicate the phone is voiced
        data=np.append(mfccs,0)
 
        writer.writerow(data)

# For voicedless data
for fn in os.listdir(voicedlessPath):
    # Subsample dataset, retrieve 1 in 10 among dataset
    randNum=np.random.randint(10)
    if randNum !=0:
        continue
    
    # Read wav file as "sig"
    fileName,ext=os.path.splitext(fn)
    wavFile=voicedlessPath+fileName+".wav"
    sig,samplerate=sf.read(wavFile)

    # Cut head and tail of wav file is those sections are not reliable
    status,sig=head_tail_scissor(sig)
    if status==False:
        continue

    # Write result in a csv file
    with open("MFCC_Diag.csv","a",newline="") as csvfile:
        writer=csv.writer(csvfile)

        # MFCC
        mfccs = librosa.feature.mfcc(y=sig, sr=samplerate, n_mfcc=10, n_fft=int(len(sig)/2),hop_length=int(len(sig)/4))
        mfccs=np.abs(mfccs.flatten())

        # Add last feature to indicate if it is voiced/ voicedless
        # 1 indicate the phone is voicedless
        data=np.append(mfccs,1)
 
        writer.writerow(data)


/opt/anaconda3/lib/python3.12/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [2]:
# Read the csv file into DataFrame
df=pd.read_csv('MFCC_Diag2.csv',header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,265.228042,458.186801,470.257051,457.539243,414.467666,90.614599,192.057107,169.739043,182.347215,124.984435,...,34.043803,13.143659,6.325329,5.132010,5.759157,15.230839,6.594397,15.439220,1.799922,0.0
1,328.256481,405.960194,449.699043,466.053159,418.932529,134.976019,115.128203,120.315589,132.297699,123.460497,...,24.839330,12.755944,5.231572,6.221648,14.479479,9.688808,21.666932,16.535213,12.599174,0.0
2,189.458662,213.702699,154.809471,261.947298,284.268398,115.657810,69.996766,151.142941,256.189000,165.222968,...,37.805068,35.576028,27.023775,23.136907,0.553415,27.207826,23.107657,20.735317,6.485982,0.0
3,363.786789,453.016668,526.722972,537.387478,432.480275,159.443723,212.198590,184.020792,194.517498,122.182096,...,3.364365,9.175116,23.532383,4.153646,9.437978,5.176119,13.842966,8.270917,1.559208,0.0
4,388.051051,443.337599,442.930852,454.286049,411.327413,140.440154,172.193514,182.060323,186.364496,131.781575,...,43.460843,53.283936,52.999581,37.855044,0.035997,9.076195,11.489541,11.492886,10.553542,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25731,325.944517,251.117985,239.103724,278.546103,327.691483,31.351452,51.862406,29.589785,1.941170,90.516666,...,28.802349,29.613620,24.285473,5.866695,0.111092,7.195746,0.178734,0.856633,2.610127,1.0
25732,282.071298,267.683100,301.200456,310.001145,310.962592,52.540922,18.378879,19.783781,33.093583,8.130309,...,4.178397,15.260363,21.261043,22.388343,4.899059,5.245044,9.066780,5.972529,2.437430,1.0
25733,302.418172,276.837027,309.440976,344.243584,374.389948,9.319045,107.465301,131.122728,143.962905,114.479627,...,3.905217,16.615125,25.497775,15.559251,9.407525,8.798188,9.033513,7.463898,6.708542,1.0
25734,302.613214,343.319361,296.500000,380.868613,380.155548,113.485357,24.903523,39.778277,10.827210,101.066280,...,31.062576,24.322575,24.611384,14.512271,1.077841,11.962804,2.513650,3.617262,5.353234,1.0


In [3]:
# Use UMAP to reduce the feature dimension to 2
reducer = umap.UMAP()
data = df.iloc[:,0:50]
scaled_data = StandardScaler().fit_transform(data)
embedding = reducer.fit_transform(scaled_data)
embedding.shape

(25736, 2)

In [4]:
# Read the embedded feature as DataFrame
df_feature=pd.DataFrame(embedding,columns=['feature1','feature2'])
df_feature['type']=df[50]
df_feature

,feature1,feature2,type
0,8.883599,2.761565,0.0
1,8.952702,1.531281,0.0
2,11.465335,1.191309,0.0
3,9.732270,1.966294,0.0
4,9.216735,4.409920,0.0
...,...,...,...
25731,4.174520,0.862004,1.0
25732,3.868570,2.497786,1.0
25733,-0.477730,5.263850,1.0
25734,4.667093,1.890789,1.0


In [ ]:
# Set up plot configuration
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels

# Group the data based on voiced/ voicedless
groups = df_feature.groupby('type')

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.margins(0.05)
typeDict= {1:'voicedless',0:'voiced'}
for type, group in groups:
    if type==1:
        ax.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type],alpha=0.5, color='#4d4dff')
    if type==0:
        ax.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type],alpha=0.5, color='#ff5c33')
legend=ax.legend(fontsize=15,markerscale=4,loc='upper center')
plt.xlabel('UMAP_1',fontsize=15)
plt.ylabel('UMAP_2',fontsize=15)

## Save figure as pdf file
#plt.savefig("figure/featureAna_mfcc1.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# Plot individual 
plt.figure(figsize=(6, 3))

for type, group in groups:
    if type==0:
        plt.subplot(1, 2, 1)
        plt.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type], alpha=0.5, color='#ff5c33')
        plt.legend(['voiced'],fontsize=10,markerscale=4,loc='upper center')
        plt.xlabel('UMAP_1')
        plt.ylabel('UMAP_2')
        plt.xlim([-2.2, 13])
        plt.ylim([-2.2, 8.5])
    if type==1:
        plt.subplot(1, 2, 2)
        plt.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type], alpha=0.5, color='#4d4dff')
        plt.legend(['voicedless'],fontsize=10,markerscale=4,loc='upper center')
        plt.xlabel('UMAP_1')
        plt.ylabel('UMAP_2')
        plt.xlim([-2.2, 13])
        plt.ylim([-2.2, 8.5])

plt.tight_layout()
plt.show()

## Save figure as pdf file
#plt.savefig("figure/featureAna_mfcc2.pdf", format="pdf", bbox_inches="tight")